In [1]:
import os
import librosa
import soundfile
import numpy as np
import pandas as pd
from os.path import join, exists
from tqdm import tqdm

In [2]:
datasets = ["freesound"]
raw_data = join("data", "raw_data")
clean_data = join("data", "clean_data")
sr = 44100
sample_duration = 0.96

In [ ]:
sample_n = 0
for dataset in datasets:
    result = []
    csv = pd.read_csv(join(raw_data, dataset, "samples.csv"), index_col=0)
    for i, row in tqdm(csv.iterrows()):
        file, label = row
        x, orig_sr = librosa.load(join(raw_data, dataset, "samples", file), sr=sr)
        x = librosa.resample(x, orig_sr, sr)
        x, _ = librosa.effects.trim(x, top_db=35)
        n_slices = int(np.ceil(x.shape[0] / sr))
        for i in range(n_slices):
            sample_name = f"{sample_n}.wav"
            sample = x[i * sr:(i + 1) * sr]
            soundfile.write(join(clean_data, dataset, "samples", sample_name), sample, sr)
            result.append((sample_name, label))
            sample_n += 1
    df = pd.DataFrame(result, columns=["file", "label"])
    df.to_csv(join(clean_data, dataset, "samples.csv"))

213it [01:09,  1.71it/s]